In [11]:
import pandas as pd

In [13]:
uhauldata = pd.read_csv("../data/uhauloutput-35cities.csv")

In [18]:
incomedata = pd.read_csv("../data/income.csv")

In [21]:
incomedata = incomedata.drop("Unnamed: 0", axis=1)

In [22]:
uhauldata_filtered = uhauldata[
    (uhauldata['Pickup Location'].isin(incomedata["City"].unique())) &
    (uhauldata['Dropoff Location'].isin(incomedata["City"].unique()))
]

In [23]:
uhauldata_filtered

,Pickup Location,Dropoff Location,Rate,Truck Type,Moving Type,Date
0,"Atlanta, GA","Boston, MA","$2,111.00",10' Truck,Studio to 1 Bedroom Apt.,09/28/2024
1,"Atlanta, GA","Boston, MA","$2,223.00",15' Truck,1 Bedroom Home to 2 Bedroom Apt.,09/28/2024
2,"Atlanta, GA","Boston, MA","$2,443.00",20' Truck,2 Bedroom Home to 3 Bedroom Apt.,09/28/2024
3,"Atlanta, GA","Boston, MA","$2,779.00",26' Truck,3 Bedroom Home to 4 Bedroom Home,09/28/2024
8,"Atlanta, GA","Charlotte, NC",$429.00,10' Truck,Studio to 1 Bedroom Apt.,09/28/2024
...,...,...,...,...,...,...
4736,"Wichita, KS","Tulsa, OK",$375.00,26' Truck,3 Bedroom Home to 4 Bedroom Home,09/28/2024
4737,"Wichita, KS","Washington, DC","$1,749.00",10' Truck,Studio to 1 Bedroom Apt.,09/28/2024
4738,"Wichita, KS","Washington, DC","$1,841.00",15' Truck,1 Bedroom Home to 2 Bedroom Apt.,09/28/2024
4739,"Wichita, KS","Washington, DC","$2,209.00",20' Truck,2 Bedroom Home to 3 Bedroom Apt.,09/28/2024


In [33]:
uhauldata_filtered['Rate'] = uhauldata_filtered['Rate'].str.replace('$', '', regex=False)


/var/folders/_k/twyqm8dd3gvdw3vfzr0hk_5r0000gn/T/ipykernel_75888/849234699.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uhauldata_filtered['Rate'] = uhauldata_filtered['Rate'].str.replace('$', '', regex=False)


In [41]:
uhauldata_filtered['Rate'] = uhauldata_filtered['Rate'].replace(',', '', regex=True).astype(float)

/var/folders/_k/twyqm8dd3gvdw3vfzr0hk_5r0000gn/T/ipykernel_75888/3529183148.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uhauldata_filtered['Rate'] = uhauldata_filtered['Rate'].replace(',', '', regex=True).astype(float)


In [44]:
rate_sum_by_ending_city = uhauldata_filtered.groupby('Dropoff Location')['Rate'].mean().reset_index()
rate_sum_by_ending_city

,Dropoff Location,Rate
0,"Albuquerque, NM",1696.903226
1,"Atlanta, GA",1639.752066
2,"Austin, TX",1840.911290
3,"Baltimore, MD",1789.062016
4,"Boston, MA",2094.769231
5,"Charlotte, NC",1938.523438
6,"Chicago, IL",1564.512195
7,"Cincinnati, OH",1458.453125
8,"Cleveland, OH",1571.585938
9,"Columbus, OH",1525.781250


In [46]:
truckindex = pd.read_csv("../data/truckindex.csv")


In [47]:
merged_df = rate_sum_by_ending_city.assign(key=1).merge(truckindex.assign(key=1), on='key').drop('key', axis=1)

In [48]:
merged_df['TruckingCost'] = merged_df['Rate'] * merged_df['PriceIndex']

In [52]:

# Strip whitespace from column names (if any)
merged_df.columns = merged_df.columns.str.strip()

# Step 4: Create Final DataFrame with trucking costs
final_df = merged_df[['Dropoff Location', 'Year', 'TruckingCost']]

In [53]:
final_df

,Dropoff Location,Year,TruckingCost
0,"Albuquerque, NM",2005,2110.947613
1,"Albuquerque, NM",2006,2183.914452
2,"Albuquerque, NM",2007,2136.401161
3,"Albuquerque, NM",2008,1914.106839
4,"Albuquerque, NM",2009,1568.787032
...,...,...,...
622,"Wichita, KS",2019,1875.071344
623,"Wichita, KS",2020,1730.099813
624,"Wichita, KS",2021,1899.764297
625,"Wichita, KS",2022,1916.491781


In [54]:
final_df.to_csv("../data/truckcost.csv")

In [22]:
import pandas as pd
df = pd.read_csv("../data/predictions.csv")
data2024 = df[df["year"] == 2024].reset_index()
data2026 = df[df["year"] == 2026].reset_index()
data2024 = data2024.drop(["index", "Unnamed: 0"], axis=1)
data2026 = data2026.drop(["index", "Unnamed: 0"], axis=1)

In [35]:
data2024['city'].unique()

array(['Albuquerque, NM', 'Atlanta, GA', 'Austin, TX', 'Baltimore, MD',
       'Boston, MA', 'Charlotte, NC', 'Chicago, IL', 'Cincinnati, OH',
       'Cleveland, OH', 'Columbus, OH', 'Dallas, TX', 'Denver, CO',
       'Detroit, MI', 'Fresno, CA', 'Houston, TX', 'Jacksonville, FL',
       'Memphis, TN', 'Milwaukee, WI', 'Minneapolis, MN', 'Omaha, NE',
       'Orlando, FL', 'Philadelphia, PA', 'Phoenix, AZ', 'Pittsburgh, PA',
       'Portland, OR', 'Raleigh, NC', 'Sacramento, CA', 'Seattle, WA',
       'Tampa, FL', 'Tucson, AZ', 'Tulsa, OK', 'Washington, DC',
       'Wichita, KS'], dtype=object)

In [44]:
data2026["predicted_housing_price"] - data2024["predicted_housing_price"]

0      63901.19
1      86651.48
2     108443.85
3      64844.19
4     100229.80
5      76423.50
6      49001.70
7      51669.98
8      44754.10
9      60213.00
10     68663.44
11    104422.40
12     39745.81
13     78563.94
14     60292.91
15     72316.82
16     42806.76
17     56195.69
18     59820.28
19     55762.75
20     82301.94
21     64167.41
22     90735.60
23     48074.50
24     97447.90
25     90867.60
26     74987.18
27     53609.50
28     57091.60
29     54154.00
30     64269.59
31     66761.10
32     56549.84
Name: predicted_housing_price, dtype: float64

In [45]:
data2024

,city,year,predicted_income,predicted_mean_age,predicted_unemployment,predicted_truck_cost,predicted_geomobility,predicted_housing_price
0,"Albuquerque, NM",2024,70163.090,38.948875,4.580811,1959.4790,529106.90,330278.34
1,"Atlanta, GA",2024,87295.840,34.588352,4.297268,1905.9230,515154.25,389576.30
2,"Austin, TX",2024,94406.620,35.071190,2.837417,2145.7580,990303.25,552833.25
3,"Baltimore, MD",2024,62932.690,36.753254,5.474064,2063.4094,552534.44,388377.75
4,"Boston, MA",2024,95382.310,33.906580,4.970753,2435.2130,652547.00,642227.00
5,"Charlotte, NC",2024,79774.540,34.951550,2.990472,2223.8290,920345.44,379209.60
6,"Chicago, IL",2024,76770.945,36.511604,6.079724,1783.5789,2624937.50,307968.60
7,"Cincinnati, OH",2024,56025.246,33.860280,6.179176,1722.2528,304840.12,286020.72
8,"Cleveland, OH",2024,39142.610,36.864544,7.679967,1822.1686,383038.12,224159.20
9,"Columbus, OH",2024,65840.750,33.341763,4.516364,1761.9917,906606.40,318601.10


In [40]:
result = (data2026["predicted_housing_price"] - data2024["predicted_housing_price"])/data2024["predicted_housing_price"]*100

In [41]:
new_df = pd.DataFrame({
    'city': data2024['city'].unique(),  # Assuming 'city' is the column with city names
    'ROI': result
})

In [42]:
new_df

,city,ROI
0,"Albuquerque, NM",19.347678
1,"Atlanta, GA",22.242493
2,"Austin, TX",19.616014
3,"Baltimore, MD",16.696165
4,"Boston, MA",15.606600
5,"Charlotte, NC",20.153366
6,"Chicago, IL",15.911265
7,"Cincinnati, OH",18.065118
8,"Cleveland, OH",19.965319
9,"Columbus, OH",18.899181


In [46]:
new_df.to_csv("../data/ROI.csv")